In [1]:
import re 
import csv 
from getpass import getpass 
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge, EdgeOptions

In [2]:
options = EdgeOptions()
options.use_chromium = True 

PATH = "C:\Program Files (x86)\\msedgedriver.exe"
driver = Edge(options=options, executable_path=PATH)

driver.get('https://www.twitter.com/login')
sleep(5)

# login manually

In [3]:
search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input.send_keys('kampus seksual')
search_input.send_keys(Keys.RETURN)
sleep(1)

driver.find_element_by_link_text('Latest').click()

In [3]:
def get_tweet_data(card):
    # USER MAIN INFO
    username = card.find_element_by_xpath('.//span').text
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    tweet = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text

    # TWEET STATS
    rep_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    rtwt_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    lk_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text

    reply_count = rep_cnt if rep_cnt != '' else 0
    retweet_count = rtwt_cnt if rtwt_cnt != '' else 0
    like_count = lk_cnt if lk_cnt != '' else 0
    
    # ADDITIONAL INFO
    emoji_tags = card.find_elements_by_xpath('.//img[contains(@src, "emoji")]')
    emoji_list = []
    for tag in emoji_tags:
        filename = tag.get_attribute('src')
        try:
            emoji = chr(int(re.search(r'svg\/([a-z0-9]+)\.svg', filename).group(1), base=16))
        except AttributeError:
            continue
        if emoji:
            emoji_list.append(emoji)
    emojis = ' '.join(emoji_list)
    
    tweet = [username, handle, postdate, tweet, emojis, reply_count, retweet_count, like_count]
    return tweet 

In [5]:
data = []
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

data_counter = 0
while scrolling and data_counter <= 5000:
    page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    for card in page_cards:
        tweet = get_tweet_data(card)
        if tweet:
            data.append(tweet)
            data_counter += 1
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break
    
    sleep(1)

# close the web driver
driver.close()

In [7]:
import pandas as pd 
header = ['account_name', 'username', 'post_date', 'tweet', 'emojis', 'reply_count', 'retweet_count', 'like_count']
df = pd.DataFrame(data, columns=header)
df.to_csv('ks_kampus.csv')